# Peer-graded Assignment: The Battle of Neighborhoods

## Objective: define good location for a new business in Frankfurt am Main, Germany

## 1) Description of the problem and a discussion of the background

Frankfurt at Main is known as a very international city in Germany. The developed bank and finance sectors continuously attract skilled workers from abroad. As a consequence, demands on other sectors such as hotels, properties, restaurants, entertainment, and transportation are also increasing and prices are skyrocketing as a consequence. 

To support these changes, the city is opening credit lines for local entrepreneurs to invest in these sectors and enjoy this economical growth.

Observing a good opportunity and knowing that the German cuisine is not quite appealing for foreigners, a skilled Brazilian chef de cuisine decided to apply for the available credit line and also invest all his savings in opening a Philippine restaurant. 

However, the soon to be a restaurant owner is aware of the challenges ahead and given that I) the target clients are very demanding, II) the restaurant location has to be close to the finance and banking area, and III) that the prices of renting in the city are soaring, the business location is crucial for its long-term success.

#### The chef de cuisine hired a data scientist to locate good place for the business. The restaurant has to be within walking distance from the center of the financing neighborhood, close to popular attractions, and avoid being close to other restaurants.

## 2) Description of the data and how it will be used to solve the problem

The target area for the new business location is Frankfurt's banking neighborhood.

The analysis will require Foursquare API to retrieve information about the location, rating, description, and popularity of restaurants and attractions in the area.

The client gave the following rules and recommendations: 

1) The new location has to be at a maximum of 1 km away from the Commerzbank Tower.

2) The new location has to be at a minimum of 200 meters away from any other restaurant and 400 meters away to any other Asian restaurant, such as Japanese, Thai, Vietnamese or Indian.

3) If possible, close to the Main River.

#### After obtaining and cleaning the dataset, we are going to apply classification models and GIS analysis to determine hotspots for the restaurant implementation.

#### Let's have a sneak preview of the data:

In [1]:
import requests
import pandas as pd
from pandas import json_normalize

# Frankfurt's finance center lat / lon
fkf_latlon=[50.110924,8.682127]

# Foursquare API credentials are stored in a hidden file
from secret_settings import * 

# Foursquare API request parameters
LIMIT = 100 
radius = 2500

# request from Foursquare API
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(    
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    fkf_latlon[0], 
    fkf_latlon[1], 
    radius, 
    LIMIT)

# get data using requests
raw_data = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter columns
venues = raw_data['response']['groups'][0]['items']  
fkf_location_venues = json_normalize(venues)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
fkf_location_venues =fkf_location_venues.loc[:, filtered_columns]

# filter the category for each row
fkf_location_venues['venue.categories'] = fkf_location_venues.apply(get_category_type, axis=1)

# clean columns
fkf_location_venues.columns = [col.split(".")[-1] for col in fkf_location_venues.columns]
fkf_location_venues.head()

,name,categories,lat,lng
0,Kleinmarkthalle,Market,50.112778,8.682958
1,Góc Phố,Vietnamese Restaurant,50.113509,8.681686
2,Weinterasse Rollanderhof,Wine Bar,50.112473,8.682164
3,Römerberg,Plaza,50.110489,8.682131
4,SCHIRN Kunsthalle,Art Museum,50.110291,8.683542


## 3) Quick preview on the city bank & finance area

In [2]:
import folium
# create research area map 
city_map = folium.Map(location=[fkf_latlon[0],fkf_latlon[1]], zoom_start=12)

# add a circle with radius 2 km around our center point
folium.Circle([fkf_latlon[0],fkf_latlon[1]],
                    radius=1000
                   ).add_child(folium.Popup('Research area - 1 km radius from Commerzbank Tower')).add_to(city_map)

# call the map
city_map.save('city_map.html')
city_map

Click [here](city_map.html) if the map doesn't display - GITHUB doesn't render Jupyter Notebook maps properly.

In [3]:
# create research area map with markers
city_map_marked = folium.Map(location=[fkf_latlon[0],fkf_latlon[1]], zoom_start=15)

# add a circle with radius 2 km around our center point
folium.Circle([fkf_latlon[0],fkf_latlon[1]],
                    radius=1000
                   ).add_child(folium.Popup('Research area - 1 km radius from Commerzbank Tower')).add_to(city_map_marked)

## create list of markers
map_label = fkf_location_venues[['categories']].values.tolist()
map_latlon = fkf_location_venues[['lat','lng']].values.tolist()

# add markers 
for point in range(0, len(map_latlon)):
    folium.Marker(map_latlon[point],  popup=map_label[point]).add_to(city_map_marked)

city_map_marked.save('city_map_marked.html')
city_map_marked

Click [here](city_map_marked.html) if the map doesn't display - GITHUB doesn't render Jupyter Notebook maps properly.

#### Create subcategories for the venues 

In [47]:
# create a series with all venues types
venues_type=fkf_location_venues['categories'].value_counts()

# create a dataframe with all restaurants
fkf_restaurants=fkf_location_venues[fkf_location_venues['categories'].str.contains("Restaurant")]

# add column to check if restaurant is asian
asian_specific=['Vietnamese','Sushi','Thai','Korean','Chinese','Japanese']
pattern = '|'.join(asian_specific)
check_asian=(fkf_restaurants.categories.str.contains(pattern))
check_asian_df = pd.DataFrame({'asian': check_asian})
fkf_restaurants=fkf_restaurants.join(check_asian_df)


In [55]:
# create research area map with markers
city_map_marked_res_buffer = folium.Map(location=[fkf_latlon[0],fkf_latlon[1]], zoom_start=15)

# create list of markers
map_label = fkf_restaurants[['name','categories','asian']].values.tolist()
map_latlon = fkf_restaurants[['lat','lng']].values.tolist()

# add a circle with radius 1 km around our center point
folium.Circle([fkf_latlon[0],fkf_latlon[1]],
                    radius=1000
                   ).add_child(folium.Popup('Research area')).add_to(city_map_marked_res_buffer)

# add markers 
for point in range(0, len(map_latlon)):
    # add a circle with radius 200 or 400 km around our center point
    if map_label[point][2]:
        folium.Circle(map_latlon[point],
                        radius=400,
                        fill=True,
                        color='#f78080',
                        opacity=0.1,
                        fill_opacity=0.5
                       ).add_child(folium.Popup('Asian restaurant radius')).add_to(city_map_marked_res_buffer)
    else:
        folium.Circle(map_latlon[point],
                        radius=200,
                        fill=True,
                        color='#85ace4',
                        opacity=0.1,
                        fill_opacity=0.5
                       ).add_child(folium.Popup('Non Asian restaurant radius')).add_to(city_map_marked_res_buffer)

city_map_marked.save('city_map_marked_res_buffer.html')
city_map_marked_res_buffer

In [10]:
# verify venue categories, group them and do some histogram analysis etc

In [51]:
map_label = (fkf_restaurants[['asian']]).values.tolist()
map_label

[[True],
 [True],
 [False],
 [False],
 [False],
 [True],
 [False],
 [False],
 [False],
 [False],
 [True],
 [False],
 [False],
 [False],
 [False],
 [False],
 [True],
 [False],
 [True],
 [False],
 [False]]

In [45]:
fkf_restaurants

,name,categories,lat,lng
1,Góc Phố,Vietnamese Restaurant,50.113509,8.681686
12,Superkato,Sushi Restaurant,50.111664,8.679153
14,Heimat – Essen und Weine,German Restaurant,50.111125,8.678286
22,Seven Swans & The Tiny Cup,Vegetarian / Vegan Restaurant,50.109534,8.686896
29,Emir‘Et,Turkish Restaurant,50.113518,8.689303
30,Kabuki,Japanese Restaurant,50.109035,8.670183
52,Club Michel,Modern European Restaurant,50.108183,8.670867
65,Saravanaa Bhavan,Indian Restaurant,50.108160,8.666834
67,Hummus Küch',Falafel Restaurant,50.101932,8.687351
70,a tavola!,Italian Restaurant,50.101627,8.681500


Click [here](city_map_marked_res.html) if the map doesn't display - GITHUB doesn't render Jupyter Notebook maps properly.